# How to access and display a Sentinel-2 image through GEE, analyze the spectral band intensities for areas of interest, and segment the image with SVM, XGBOOST, Random Forests, or K-means clustering

### Table of Content:

[Part1: search for Sentinel-2 images through GEE and save as GeoTIFs](#-Part1:-search-for-Sentinel-2-images-through-GEE-and-save-as-GeoTIFs) <br>
[Part2: read the files again in with rasterio and characterize band intensities](#-Part2:-read-the-files-again-in-with-rasterio-and-characterize-band-intensities) <br>
[Part 3: Pixel-level classification with your bands](#-Part-3:-Pixel-level-classification-with-your-bands) <br>

In [ ]:
### install necessary libraries

#! pip install -U scikit-learn
#! pip install rasterio
#! pip install matplotlib shapely rich holoviews rioxarray
#! pip install earthengine-api
#! pip install geemap
#! pip install scikit-learn-intelex
#! pip install tifffile
#! pip install scikit-image
#! pip install opencv-python
#! pip install pykml

# Part1: search for Sentinel-2 images through GEE and save as GeoTIFs

In [ ]:
### import necessary libraries

import ee
import geemap
import sklearn
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import fiona

#### log into google earth-engine 
For this you need an account with gee and you need to have created a project. The name of the project you assign the your_authentication_gee_project variable 

In [ ]:
your_authentication_gee_project = "dp-ee-omdena" # add your project name here

ee.Authenticate()
ee.Initialize(project=your_authentication_gee_project)

#### define area of interest by reading in a geojson
alternatively look up coordinated directly here: https://geojson.io/#map=2/0/20
and pluf into: 
AOI = ee.Geometry.Polygon([[ COORDINATES ]])

Akis Northern Sardinia (AOI1):

In [ ]:
# Path to your GeoJSON file
geojson_file_path = './Akis/sardinia_aoi_2.geojson'

# Load GeoJSON as an ee.Geometry
AOI1 = geemap.geojson_to_ee(geojson_file_path)

Greek Beach - Eastern Macedonia (AOI2):

In [ ]:
### from Dagshub file 'eastern_macedonian_coast_greek_sea.geojson'

AOI2 = ee.Geometry.Polygon([[
      [24.119557891949093, 40.657321340525456],
      [24.365906219911057, 40.77346272940198],
      [24.293121796082932, 40.84570271430106],
      [24.050049286317307, 40.7344516585937],
      [24.119557891949093, 40.657321340525456]]])

Greek Beach - Lourdas Bay (AOI3):

In [ ]:
### from Dagshub file 'lourdas_bay_greek_sea.geojson'

AOI3 = ee.Geometry.Polygon([[
    [20.667496, 38.131663], 
    [20.667496, 38.051675], 
    [20.565864, 38.051675], 
    [20.565864, 38.131663], 
    [20.667496, 38.131663]]])

Mallorquin coast - Muntanyes d' Arta (AOI4):

In [ ]:
### Mallorca file 'Muntanyes_d_Arta.kml'

# Set driver support for KML files
fiona.drvsupport.supported_drivers['KML'] = 'rw'

kml_file_1 = './GeoTIFsForSVM/Muntanyes_d_Arta.kml'

# Read the KML file into a GeoDataFrame
gdf_Mallorca_1 = gpd.read_file(kml_file_1, driver='KML')

In [ ]:
from shapely.geometry import Polygon

# Convert geometries to 2D (remove Z coordinate)
gdf_Mallorca_1.geometry = gdf_Mallorca_1.geometry.apply(lambda geom: Polygon([(point[0], point[1]) for point in geom.exterior.coords]))

# Check the geometries again
gdf_Mallorca_1.head(3)

In [ ]:
### get AOI for ee library

bbox1 = gdf_Mallorca_1.total_bounds

minx, miny, maxx, maxy = bbox1

# Create a bounding box geometry with the correct CRS
crs = 'EPSG:4326'  # Assuming the bounding box coordinates are in WGS84
AOI4 = ee.Geometry.Rectangle([minx, miny, maxx, maxy], crs, False)

In [ ]:
### check
from shapely.geometry import mapping
# Create a geemap Map instance
M = geemap.Map()

fc1= geemap.geopandas_to_ee(gdf_Mallorca_1)

# Add the bounding box geometry to the map
M.addLayer(AOI4, {'color': 'blue'}, 'Bounding Box Geometry')
M.centerObject(AOI4, zoom=10)
M.add_ee_layer(fc1, {'color': 'red'}, 'Polygons from GeoPandas')

# Display the map
M

Mallorquin coast - Costa de Llevant (AOI5):

In [ ]:
### Mallorca file 'Muntanyes_d_Arta.kml'

# Set driver support for KML files
fiona.drvsupport.supported_drivers['KML'] = 'rw'

kml_file_2 = './GeoTIFsForSVM/Costa de Llevant.kml'

# Read the KML file into a GeoDataFrame
gdf_Mallorca_2 = gpd.read_file(kml_file_2, driver='KML')

In [ ]:
from shapely.geometry import Polygon

# Convert geometries to 2D (remove Z coordinate)
gdf_Mallorca_2.geometry = gdf_Mallorca_2.geometry.apply(lambda geom: Polygon([(point[0], point[1]) for point in geom.exterior.coords]))

# Check the geometries again
gdf_Mallorca_2.head(3)

In [ ]:
### get AOI for ee library

bbox2 = gdf_Mallorca_2.total_bounds

minx, miny, maxx, maxy = bbox2

# Create a bounding box geometry with the correct CRS
crs = 'EPSG:4326'  # Assuming the bounding box coordinates are in WGS84
AOI5 = ee.Geometry.Rectangle([minx, miny, maxx, maxy], crs, False)

In [ ]:
### check

# Create a geemap Map instance
M3 = geemap.Map()
fc2= geemap.geopandas_to_ee(gdf_Mallorca_2)

# Add the bounding box geometry to the map
M3.addLayer(AOI5, {'color': 'blue'}, 'Bounding Box Geometry')
M3.centerObject(AOI5, zoom=10)
M3.add_ee_layer(fc2, {'color': 'red'}, 'Polygons from GeoPandas')
# Display the map
M3

#### search collection and do cloud filtering on the fly 

In [ ]:
### chose image collectionw

s2SrCollection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')

In [ ]:
### some  cloud correction thresholds

### do cloud correction:
ROI = AOI2

QA_BAND = 'cs'
CLEAR_THRESHOLD = 0.80

In [ ]:
### get actual image collection

composite = s2SrCollection.filterBounds(ROI).filterDate('2022-01-01', '2022-02-01').linkCollection(csPlus, [QA_BAND]).map(lambda img: img.updateMask(img.select(QA_BAND).gte(CLEAR_THRESHOLD)))

In [ ]:
### check how many images were selected

composite.size()

#### save ROI as shapefile

In [ ]:
# Convert the Earth Engine geometry to a feature
feature_ROI = ee.Feature(ROI, {})

# Create a feature collection
fc_ROI = ee.FeatureCollection([feature_ROI])

# Specify the path to save the shapefile
output_shp = './GeoTIFsForSVM/AOI2_shapefile.shp'

# Export the feature collection as a shapefile
geemap.ee_export_vector(fc_ROI, filename=output_shp)

#### create median value image

In [ ]:
image = composite.median()

#### mask the land 

In [ ]:
def mask_land(image):
    # This is a simple example. You may need a more specific way to identify water vs. land.
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # NDWI (Normalized Difference Water Index) calculation
    water_threshold = 0  # Threshold value to distinguish water; adjust as needed -> some say needs to be 0.3 or 0
    water_mask = ndwi.gte(water_threshold)  # Water areas
    return image.updateMask(water_mask)

In [ ]:
masked_image = mask_land(image)

In [ ]:
### check

M1 = geemap.Map()
M1.addLayer(masked_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'Masked Image')  # Visualize using RGB bands
M1.centerObject(ROI, 12)  # Zoom into the area around Corsica
M1

#### store water mask as shapefile

In [ ]:
roi = ROI

# Load an image
image2 = image

# Create your mask based on band ratios
ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')  # NDWI (Normalized Difference Water Index) calculation
water_threshold = 0  # Threshold value to distinguish water; adjust as needed -> some say needs to be 0 or 0.3
water_mask = ndwi.gte(water_threshold) 

# For example, converting a numpy array mask to an EE Image
mask_ee = ee.Image(water_mask)

# Convert the mask to vector (polygon) features
features = mask_ee.reduceToVectors(scale=30, geometry=ROI)

# Convert the vector features to a FeatureCollection
feature_collection = ee.FeatureCollection(features)

In [ ]:
### check

# Create a geemap Map instance
M5 = geemap.Map()
#fc2= geemap.geopandas_to_ee(gdf_Mallorca_2)

# Add the bounding box geometry to the map
#M5.addLayer(ROI, {'color': 'blue'}, 'Bounding Box Geometry')
M5.centerObject(ROI, zoom=10)
M5.add_ee_layer(feature_collection, {'color': 'red'}, 'Polygons from GeoPandas')
# Display the map
M5

In [ ]:
# Convert the Earth Engine geometry to a feature
#feature_ROI = ee.Feature(ROI, {})

# Create a feature collection
#fc_ROI = ee.FeatureCollection([feature_ROI])

# Specify the path to save the shapefile
output_shp2 = "./GeoTIFsForSVM/AOI2d_watermask_geojson.geojson"

# Export the feature collection as a shapefile
geemap.ee_export_vector(feature_collection, filename=output_shp2)

#### optional: calculate ratios between bands or normalized differences already with ee

In [ ]:
def addRatios(image):
    rB8B2 = image.expression(
        '(B8/B2)',
        {
            'B2': image.select('B2'),  # Change to 'B8A' for 705 nm
            'B8': image.select('B8'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB8B2')
    rB8B4 = image.expression(
        '(B8/B4)',
        {
            'B4': image.select('B4'),  # Change to 'B8A' for 705 nm
            'B8': image.select('B8'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB8B4')
    rB3B2 = image.expression(
        '(B3/B2)',
        {
            'B2': image.select('B2'),  # Change to 'B8A' for 705 nm
            'B3': image.select('B3'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB3B2')
    rB8AB8 = image.expression(
        '(B8A/B8)',
        {
            'B8A': image.select('B8A'),  # Change to 'B8A' for 705 nm
            'B8': image.select('B8'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB8AB8')
    rB3B1 = image.expression(
        '(B3/B1)',
        {
            'B3': image.select('B3'),  # Change to 'B8A' for 705 nm
            'B1': image.select('B1'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB3B1')
    rB5B4 = image.expression(
        '(B5/B4)',
        {
            'B4': image.select('B4'),  # Change to 'B8A' for 705 nm
            'B5': image.select('B5'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB5B4')
    rB4B1 = image.expression(
        '(B4/B1)',
        {
            'B4': image.select('B4'),  # Change to 'B8A' for 705 nm
            'B1': image.select('B1'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB4B1')
    rB2B1 = image.expression(
        '(B2/B1)',
        {
            'B2': image.select('B2'),  # Change to 'B8A' for 705 nm
            'B1': image.select('B1'),  # Change to 'B11' for 665 nm
        }
    ).rename('RB2B1')
    return image.addBands([rB8B2]).addBands([rB8B4]).addBands([rB3B2]).addBands([rB8AB8]).addBands([rB3B1]).addBands([rB5B4]).addBands([rB4B1]).addBands([rB2B1

In [ ]:
def addNormDiff(image):
#    rnir1 = image.normalizedDifference(['B5', 'B4']).rename('RNIR1')
#    image = image.addBands(rnir1)
    ###
    B3B2 = image.normalizedDifference(['B3', 'B2']).rename('B3B2band')
    image = image.addBands(B3B2)
    ###
    B8B2 = image.normalizedDifference(['B8', 'B2']).rename('B8B2band')
    image = image.addBands([B8B2])
    ###
    B8B4 = image.normalizedDifference(['B8', 'B4']).rename('B8B4band')    
    image = image.addBands([B8B4])
    ###
    B1B2 = image.normalizedDifference(['B1', 'B2']).rename('B1B2band')
    image = image.addBands([B1B2])
    ###
    B1B4 = image.normalizedDifference(['B1', 'B4']).rename('B1B4band')
    image = image.addBands([B1B4])
#
    B5B4 = image.normalizedDifference(['B5', 'B4']).rename('B5B4band')
    image = image.addBands([B5B4])
    B8B8A = image.normalizedDifference(['B8', 'B8A']).rename('B8B8Aband')
    image = image.addBands([B8B8A])

    return image

In [ ]:
withRatios = addRatios(masked_images)
withRatios.bandNames().getInfo()

In [ ]:
withNormDiff = addNormDiff(masked_images)
withNormDiff.bandNames().getInfo()

#### save as GeoTIF
Save as 2 files (2 each) due to size limitations of ee

In [ ]:
tofile1 = masked_image.select(['B1', 'B2', 'B3', 'B4'])

filename1 = './GeoTIFsForSVM/AOI2c_1_B1_B2_B3_B4_med_Jan_GeoTIFF.tif'
image_1 = tofile1.clip(ROI).unmask()
geemap.ee_export_image(image_1, filename=filename1, scale=30, crs='EPSG:4326', region=ROI, file_per_band=False)

In [ ]:
tofile2 = masked_image.select(['B5', 'B8', 'B8A'])

filename2 = './GeoTIFsForSVM/AOI2c_2_B5_B8_B8A_med_Jan_GeoTIFF.tif'
image_2 = tofile2.clip(ROI).unmask()
geemap.ee_export_image(image_2, filename=filename2, scale=30, crs='EPSG:4326', region=ROI, file_per_band=False)

# Part2: read the files again in with rasterio and characterize band intensities

In [ ]:
### import required libraries

import rasterio as rio
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import shape
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterio.features import geometry_mask
from rasterio.plot import show
import holoviews as hv
import hvplot.xarray
import xarray as xr
import hvplot.pandas
from shapely.geometry import box
from shapely.ops import unary_union
import os
from rasterio.enums import Resampling
import contextily
from rasterio.mask import mask
import subprocess
import rasterio as rio
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona

#### read in seagrass polygons

#### from Akis seagrass polygon files (AOI1)

In [ ]:
### load mask

import glob

# Specify the path to the folder containing GeoJSON files
folder_path = "./Akis/features/"

# Get a list of all GeoJSON files in the folder
geojson_files = glob.glob(folder_path + "*.geojson")

# Initialize an empty GeoPandas DataFrame
gdf_list = []

# Loop through each GeoJSON file and read into a GeoPandas DataFrame
for file_path in geojson_files:
    gdf = gpd.read_file(file_path)
    gdf_list.append(gdf)

In [ ]:
# Concatenate all GeoPandas DataFrames into a single DataFrame
combined_gdf = pd.concat(gdf_list, ignore_index=True)

# Print or do further processing with the combined GeoPandas DataFrame
print(combined_gdf.head())

In [ ]:
# Assuming combined_gdf is your GeoDataFrame
non_overlapping_geometry = combined_gdf.unary_union

# Convert the MultiPolygon to a GeoDataFrame with one row per Polygon
final_gdf = gpd.GeoDataFrame(geometry=[geom for geom in non_overlapping_geometry.geoms])
final_gdf = final_gdf.set_crs(combined_gdf.crs)

#### from Greek sea dataset: Eastern Macedonian coast (AOI2)

In [ ]:
### from Dagshub file 'eastern_macedonian_coast_greek_sea.geojson'

aoi_1 = ee.Geometry.Polygon([[
      [24.119557891949093, 40.657321340525456],
      [24.365906219911057, 40.77346272940198],
      [24.293121796082932, 40.84570271430106],
      [24.050049286317307, 40.7344516585937],
      [24.119557891949093, 40.657321340525456]]])

In [ ]:
from shapely.geometry import shape

# Reading Greek seas meadows into Geopandas dataframe
seagrass_gdf = gpd.read_file('./GeoTIFsForSVM/Greek_Seagrass_Meadows_v0906/Greek Seagrass meadows v0906.shp')

roi_crs = 'EPSG:4326'

# Convert EE geometry to GeoJSON
ee_geojson1 = ee.Feature(aoi_1).geometry().getInfo()

# Convert GeoJSON to Shapely geometry
shapely_geometry1 = shape(ee_geojson1)

# Create a GeoPandas GeoDataFrame
roi_1_gdf = gpd.GeoDataFrame(geometry=[shapely_geometry1], crs=roi_crs)

# Changing the CRS of Landsat .shp data from EPSG:32635 to EPSG:4326
seagrass_gdf = seagrass_gdf.to_crs(roi_crs)

# Extracting the seagrass polygons in the region of interest
seagrass_in_roi_1 = gpd.sjoin(seagrass_gdf, roi_1_gdf, how='inner', op='intersects')
#seagrass_in_roi_1.drop(['index_right', 'name'], axis=1, inplace=True)

In [ ]:
seagrass_in_roi_1

#### from Greek sea dataset: Lourdas Bay (AOI3)

In [ ]:
### from Dagshub file 'lourdas_bay_greek_sea.geojson'

aoi_2 = ee.Geometry.Polygon([[
    [20.667496, 38.131663], 
    [20.667496, 38.051675], 
    [20.565864, 38.051675], 
    [20.565864, 38.131663], 
    [20.667496, 38.131663]]])

In [ ]:
from shapely.geometry import shape

# Reading Greek seas meadows into Geopandas dataframe
seagrass_gdf = gpd.read_file('./GreekPolygons/GreekSeagrass_L8.shp')

roi_crs = 'EPSG:4326'

# Convert EE geometry to GeoJSON
ee_geojson2 = ee.Feature(aoi_2).geometry().getInfo()

# Convert GeoJSON to Shapely geometry
shapely_geometry2 = shape(ee_geojson2)

# Create a GeoPandas GeoDataFrame
roi_2_gdf = gpd.GeoDataFrame(geometry=[shapely_geometry2], crs=roi_crs)

# Changing the CRS of Landsat .shp data from EPSG:32635 to EPSG:4326
seagrass_gdf = seagrass_gdf.to_crs(roi_crs)

# Extracting the seagrass polygons in the region of interest
seagrass_in_roi_2 = gpd.sjoin(seagrass_gdf, roi_2_gdf, how='inner', op='intersects')
seagrass_in_roi_2.drop(['index_right', 'name'], axis=1, inplace=True)

#### read in polygons from 'Muntanyes_d_Arta.kml' (Mallorca) (AOI4)

In [ ]:
# Set driver support for KML files
fiona.drvsupport.supported_drivers['KML'] = 'rw'

# Specify the path to the KML file
kml_file_1 = './GeoTIFsForSVM/Muntanyes_d_Arta.kml'

# Read the KML file into a GeoDataFrame
gdf_Mallorca_1 = gpd.read_file(kml_file_1, driver='KML')

In [ ]:
from shapely.geometry import Polygon

# Convert geometries to 2D (remove Z coordinate)
gdf_Mallorca_1.geometry = gdf_Mallorca_1.geometry.apply(lambda geom: Polygon([(point[0], point[1]) for point in geom.exterior.coords]))

# Check the geometries again
gdf_Mallorca_1.head(3)

In [ ]:
# Assuming combined_gdf is your GeoDataFrame
non_overlapping_geometry_M1 = gdf_Mallorca_1.unary_union

# Convert the MultiPolygon to a GeoDataFrame with one row per Polygon
final_gdf_Mallorca1 = gpd.GeoDataFrame(geometry=[geom for geom in non_overlapping_geometry_M1.geoms])
final_gdf_Mallorca1 = final_gdf_Mallorca1.set_crs(gdf_Mallorca_1.crs)

In [ ]:
final_gdf_Mallorca1

In [ ]:
### check
import folium
#from folium.plugins import GeoJson

# Convert GeoDataFrame to GeoJSON
gdf_json = final_gdf_Mallorca1.to_json()

# Create a Folium map centered on the average of the geometry coordinates
map_center = [final_gdf_Mallorca1.geometry.centroid.y.mean(), final_gdf_Mallorca1.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add the GeoJSON data to the map
geojson_layer = folium.GeoJson(gdf_json)
m.add_child(geojson_layer)

# Display the map
m.save('map_Pos_1.html')

#### read in aoi for Akis Sardinia dataset again

In [ ]:
# Load GeoJSON file
geojson_path = './Akis/sardinia_aoi_2.geojson'
gdf = gpd.read_file(geojson_path)

# Extract the polygon from the GeoDataFrame
aoi_polygon = gdf['geometry'].iloc[0]
aoi_polygon

#### read in aoi for other regions

In [ ]:
# Load shape file
shape_path = './GeoTIFsForSVM/AOI2_shapefile.shp'
gdf_sh = gpd.read_file(shape_path)

# Extract the polygon from the GeoDataFrame
aoi_polygon = gdf_sh['geometry'].iloc[0]
aoi_polygon

#### Read in shapefile of water mask

In [ ]:
# Specify the path to the shapefile
shapefile_path = "./GeoTIFsForSVM/AOI2d_watermask_geojson.geojson"


# Read the shapefile into a GeoDataFrame
gdf_land_mask = gpd.read_file(shapefile_path)
gdf_land_mask

In [ ]:
# Calculate the area of each polygon
gdf_land_mask['area'] = gdf_land_mask.geometry.area

# Sort the DataFrame by area in descending order
gdf_sorted = gdf_land_mask.sort_values(by='area', ascending=False)

# Keep only the largest polygon (first row)
largest_polygon = gdf_sorted.iloc[0]

# Create a new GeoDataFrame with only the largest polygon
gdf_largest = gpd.GeoDataFrame(geometry=[largest_polygon.geometry])

# Set the CRS if needed
gdf_largest.crs = gdf_land_mask.crs

In [ ]:
### check
import folium
#from folium.plugins import GeoJson

# Convert GeoDataFrame to GeoJSON
gdf_json = gdf_largest.to_json()

# Create a Folium map centered on the average of the geometry coordinates
map_center = [gdf_largest.geometry.centroid.y.mean(), gdf_largest.geometry.centroid.x.mean()]
m = folium.Map(location=map_center, zoom_start=10)

# Add the GeoJSON data to the map
geojson_layer = folium.GeoJson(gdf_json)
m.add_child(geojson_layer)

# Display the map
m.save('map_land3.html')


#### Option 1: read files as they are and create rasterio mask

In [ ]:
# Replace 'your_multiband_geotiff.tif' with the actual file path
file_path = './GeoTIFsForSVM/AOI2c_1_B1_B2_B3_B4_med_Jan_GeoTIFF.tif'

# Open the GeoTIFF file with rasterio
with rio.open(file_path) as src:
    # Read the geospatial metadata
    geotransform = src.transform
    projection = src.crs
    # Read all bands as a numpy array
    bands_data = []
    for i in range(1, src.count + 1):
        band_data = src.read(i)
        bands_data.append(band_data)
    
    # Stack bands into one numpy array
    stacked_data = np.stack(bands_data, axis=-1)
    # Generate variables later needed for creating masks
    _, cropped_transform1 = mask(src, [mapping(aoi_polygon)], crop=False)
    blue_height1, blue_width1 = src.shape    


In [ ]:
stacked_data.shape

In [ ]:
import numpy as np

# Count the total number of elements in the array
total_elements = np.prod(stacked_data.shape)

# Count the number of NaN values in the array
nan_count = np.sum(np.isnan(stacked_data))

# Calculate the percentage of NaN values
percentage_nan = (nan_count / total_elements) * 100

print(f"Percentage of NaN values in stacked_data: {percentage_nan:.2f}%")

In [ ]:
# Calculate the total number of elements
total_elements = stacked_data.size

# Count zero values
num_zero_values = np.count_nonzero(stacked_data == 0)

# Calculate the percentage of zero values
percentage_zero_values = (num_zero_values / total_elements) * 100

print(f"Percentage of zero values: {percentage_zero_values:.2f}%")

In [ ]:
# Replace 'your_multiband_geotiff.tif' with the actual file path
file_path2 = './GeoTIFsForSVM/AOI2c_2_B5_B8_B8A_med_Jan_GeoTIFF.tif'

# Open the GeoTIFF file with rasterio
with rio.open(file_path2) as src2:
    # Read the geospatial metadata
    geotransform2 = src2.transform
    projection2 = src2.crs
    
    # Read all bands as a numpy array
    bands_data2 = []
    for i in range(1, src2.count + 1):
        band_data2 = src2.read(i)
        bands_data2.append(band_data2)
    
    # Stack bands into one numpy array
    stacked_data2 = np.stack(bands_data2, axis=-1)
    # Generate variables later needed for creating masks
    _, cropped_transform2 = mask(src2, [mapping(aoi_polygon)], crop=False)
    blue_height2, blue_width2 = src2.shape  

In [ ]:
# Calculate the total number of elements
total_elements = stacked_data2.size

# Count zero values
num_zero_values = np.count_nonzero(stacked_data2 == 0)

# Calculate the percentage of zero values
percentage_zero_values = (num_zero_values / total_elements) * 100

print(f"Percentage of zero values: {percentage_zero_values:.2f}%")

In [ ]:
stacked_data2.shape

In [ ]:
# Count the total number of elements in the array
total_elements = np.prod(stacked_data2.shape)

# Count the number of NaN values in the array
nan_count = np.sum(np.isnan(stacked_data2))

# Calculate the percentage of NaN values
percentage_nan = (nan_count / total_elements) * 100

print(f"Percentage of NaN values in stacked_data2: {percentage_nan:.2f}%")

#### Create land and seagrass masks

In [ ]:
### seagrass mask

seagrass_in_roi_1 = seagrass_in_roi_1.to_crs(projection) ## add needed final gdf here
#blue_height, blue_width = stacked_data[1].shape
#_, cropped_transform = mask(stacked_data[1], [mapping(aoi_polygon)], crop=False)
final_mask_Posidonia = geometry_mask(seagrass_in_roi_1.geometry, out_shape=(blue_height1, blue_width1), transform=cropped_transform1, invert=True)

In [ ]:
final_mask_Posidonia.shape

In [ ]:
import matplotlib.pyplot as plt

# Plot the binary mask
plt.imshow(final_mask_Posidonia, cmap='gray')
plt.title('Final Mask Posidonia')
plt.colorbar()
plt.show()

In [ ]:
### seagrass mask

gdf_largest = gdf_largest.to_crs(projection) ## add needed final gdf here
#blue_height, blue_width = stacked_data[1].shape
#_, cropped_transform = mask(stacked_data[1], [mapping(aoi_polygon)], crop=False)
final_mask_Land = geometry_mask(gdf_largest.geometry, out_shape=(blue_height2, blue_width2), transform=cropped_transform2, invert=True)

In [ ]:
final_mask_Land.shape

In [ ]:
import matplotlib.pyplot as plt
final_mask_Land = ~final_mask_Land # only use if needed

# Plot the binary mask
plt.imshow(final_mask_Land, cmap='gray')
plt.title('Final Mask Land')
plt.colorbar()
plt.show()

In [ ]:
############################
### important
############################

final_mask_water = ~(final_mask_Posidonia | final_mask_Land)

In [ ]:
final_mask_water.shape

In [ ]:
true_count = np.sum(final_mask_water)
print(true_count)

In [ ]:
import matplotlib.pyplot as plt

# Plot the binary mask
plt.imshow(final_mask_water, cmap='gray')
plt.title('Final Water Mask')
plt.colorbar()
plt.show()

#### Split stacked numpy arrays into single bands

In [ ]:
coastal_cropped = stacked_data[:,:,0]
blue_cropped = stacked_data[:,:,1]
green_cropped = stacked_data[:,:,2]
red_cropped = stacked_data[:,:,3]
nir_cropped = stacked_data2[:,:,1]
nir08_cropped = stacked_data2[:,:,2]
rededge1_cropped = stacked_data2[:,:,0]

In [ ]:
### check

# Count the total number of True values in the mask
total_true_values = np.sum(final_mask_Posidonia)

# Create a subarray containing only the elements corresponding to True values in the mask
subarray_values = rededge1_cropped[final_mask_Posidonia]

# Count the number of 0s in the subarray
num_zeros = np.sum(subarray_values == 0)

# Calculate the percentage of 0s relative to the total number of True values
percentage_zeros = (num_zeros / total_true_values) * 100

print(f"Percentage of 0s among the elements corresponding to True values in the mask: {percentage_zeros:.2f}%")

In [ ]:
coastal_cropped.shape

In [ ]:
print(np.isnan(green_cropped).any())

In [ ]:
nan_count = np.count_nonzero(np.isnan(coastal_cropped))

print("Number of NaN values in blue_cropped3:", nan_count)

### create numpy arrays that can be used for masking

In [ ]:
coastal_cropped2 = stacked_data[:,:,0]
blue_cropped2 = stacked_data[:,:,1]
green_cropped2 = stacked_data[:,:,2]
red_cropped2 = stacked_data[:,:,3]
nir_cropped2 = stacked_data2[:,:,1]
nir08_cropped2 = stacked_data2[:,:,2]
rededge1_cropped2 = stacked_data2[:,:,0]

coastal_cropped3 = stacked_data[:,:,0]
blue_cropped3 = stacked_data[:,:,1]
green_cropped3 = stacked_data[:,:,2]
red_cropped3 = stacked_data[:,:,3]
nir_cropped3 = stacked_data2[:,:,1]
nir08_cropped3 = stacked_data2[:,:,2]
rededge1_cropped3 = stacked_data2[:,:,0]

In [ ]:
blue_cropped4 = stacked_data[:,:,1]

In [ ]:
nir_cropped3.shape

In [ ]:
nan_count = np.count_nonzero(np.isnan(rededge1_cropped3))

print("Number of NaN values in nir_cropped3:", nan_count)

In [ ]:
nan_count = np.count_nonzero(np.isnan(blue_cropped3))

print("Number of NaN values in blue_cropped2:", nan_count)

#### Option 2 (optional): read files with a mask

In [ ]:
import rasterio as rio
from rasterio.mask import mask
from shapely.geometry import Polygon
import geopandas as gpd

# Define the path to the multiband GeoTIFF file
tif_file = 'path_to_your_multiband_image.tif'

# Convert AOI polygon to a GeoDataFrame
aoi_gdf = gpd.GeoDataFrame(geometry=[non_overlapping_geometry])

# Open the multiband GeoTIFF file using rasterio
with rio.open(tif_file) as src:
    # Get the CRS of the GeoTIFF file
    tif_crs = src.crs

    # Project AOI GeoDataFrame to the CRS of the GeoTIFF
    aoi_gdf = aoi_gdf.to_crs(tif_crs)

    # Convert AOI GeoDataFrame back to a shapely Polygon
    aoi_polygon_projected = aoi_gdf['geometry'].iloc[0]

    out_image_inside, transformed1 = mask(src, [aoi_polygon_projected], crop=True, filled=True)

    # Invert the mask to keep pixels outside the AOI
    out_image_outside, transformed2 = mask(src, [aoi_polygon_projected], crop=True, filled=True, invert=True)

    # Print dimensions of the cropped image
    print("Cropped image shape:", out_image_inside.shape)

    # Store geospatial metadata
    geospatial_metadata = src.meta

# Print the geospatial metadata
print("Geospatial metadata:", geospatial_metadata)

#### Now start calculating ratios and normalized differences for different regions (here using Option 1)

In [ ]:
blue_cropped2.shape

In [ ]:
### only look on values within seagrass polygons:

blue_cropped2[final_mask_Posidonia == False] = np.nan
green_cropped2[final_mask_Posidonia == False] = np.nan
red_cropped2[final_mask_Posidonia == False] = np.nan
nir_cropped2[final_mask_Posidonia == False] = np.nan
nir08_cropped2[final_mask_Posidonia == False] = np.nan
rededge1_cropped2[final_mask_Posidonia == False] = np.nan
coastal_cropped2[final_mask_Posidonia == False] = np.nan

In [ ]:
### Divide single bands with each other to calculate RATIOS

RB8B2 = np.divide(nir_cropped2, blue_cropped2)
RB8B4 = np.divide(nir_cropped2, red_cropped2)
RB2B1 = np.divide(blue_cropped2, coastal_cropped2)
RB3B1 = np.divide(green_cropped2, coastal_cropped2)
RB4B1 = np.divide(red_cropped2, coastal_cropped2)
RB8AB8 = np.divide(nir08_cropped2, nir_cropped2)
RB3B2 = np.divide(green_cropped2, blue_cropped2)
RB5B4 = np.divide(rededge1_cropped2, red_cropped2)

In [ ]:
### plot histograms
import matplotlib.pyplot as plt

plt.hist(RB5B4.flatten(), bins=50, color='blue', alpha=0.5, label='RB5B4')
#plt.hist(red_nir_ratio.flatten(), bins=50, color='red', alpha=0.5, label='Red/NIR Ratio')
# and so on for other ratios

plt.xlabel('Ratio Values')
plt.ylabel('Frequency')
plt.title('Histogram of Ratio Values')
plt.legend()
plt.show()

In [ ]:
print(np.nanmean(RB8B2), np.nanmean(RB8B4), np.nanmean(RB2B1), np.nanmean(RB4B1), np.nanmean(RB8AB8), np.nanmean(RB3B2), np.nanmean(RB5B4), np.nanmean(RB3B1)) 

In [ ]:
print(np.nanstd(RB8B2), np.nanstd(RB8B4), np.nanstd(RB2B1), np.nanstd(RB4B1), np.nanstd(RB8AB8), np.nanstd(RB3B2), np.nanstd(RB5B4), np.nanstd(RB3B1)) 

In [ ]:
### Calculate the NORMALIZED DIFFERENCE between bands

ndB8B2 = (nir_cropped2 - blue_cropped2) / (nir_cropped2 + blue_cropped2)
ndB8B4 = (nir_cropped2 - red_cropped2) / (nir_cropped2 + red_cropped2)
ndB2B1 = (blue_cropped2 - coastal_cropped2) / (blue_cropped2 + coastal_cropped2)
ndB3B1 = (green_cropped2 - coastal_cropped2) / (green_cropped2 + coastal_cropped2)
ndB4B1 = (red_cropped2 - coastal_cropped2) / (red_cropped2 + coastal_cropped2)
ndB8AB8 = (nir08_cropped2 - nir_cropped2) / (nir08_cropped2 + nir_cropped2)
ndB3B2 = (green_cropped2 - blue_cropped2) / (green_cropped2 + blue_cropped2)
ndB5B4 = (rededge1_cropped2 - red_cropped2) / (rededge1_cropped2 + red_cropped2)

In [ ]:
### plot histograms
import matplotlib.pyplot as plt

plt.hist(ndB5B4.flatten(), bins=50, color='blue', alpha=0.5, label='ndB5B4')
#plt.hist(red_nir_ratio.flatten(), bins=50, color='red', alpha=0.5, label='Red/NIR Ratio')
# and so on for other ratios

plt.xlabel('Ratio Values')
plt.ylabel('Frequency')
plt.title('Histogram of Ratio Values')
plt.legend()
plt.show()

In [ ]:
print(np.nanmean(ndB8B2), np.nanmean(ndB8B4), np.nanmean(ndB2B1), np.nanmean(ndB4B1), np.nanmean(ndB8AB8), np.nanmean(ndB3B2), np.nanmean(ndB5B4), np.nanmean(ndB3B1)) 

In [ ]:
print(np.nanstd(ndB8B2), np.nanstd(ndB8B4), np.nanstd(ndB2B1), np.nanstd(ndB4B1), np.nanstd(ndB8AB8), np.nanstd(ndB3B2), np.nanstd(ndB5B4), np.nanstd(ndB3B1)) 

#### look on water pixels supposedly without seagrass

In [ ]:
blue_cropped3[final_mask_water == False] = np.nan
green_cropped3[final_mask_water == False] = np.nan
red_cropped3[final_mask_water == False] = np.nan
nir_cropped3[final_mask_water == False] = np.nan
nir08_cropped3[final_mask_water == False] = np.nan
rededge1_cropped3[final_mask_water == False] = np.nan
coastal_cropped3[final_mask_water == False] = np.nan

In [ ]:
blue_cropped3[final_mask_Land == False] = np.nan

In [ ]:
total_count = np.size(blue_cropped)

print("Total number of values in blue_cropped3:", total_count)

In [ ]:
nan_count = np.count_nonzero(np.isnan(blue_cropped))

print("Number of NaN values in blue_cropped3:", nan_count)

In [ ]:
blue_cropped3.shape

In [ ]:
### Divide single bands with each other to calculate RATIOS

RB8B2 = np.divide(nir_cropped3, blue_cropped3)
RB8B4 = np.divide(nir_cropped3, red_cropped3)
RB2B1 = np.divide(blue_cropped3, coastal_cropped3)
RB3B1 = np.divide(green_cropped3, coastal_cropped3)
RB4B1 = np.divide(red_cropped3, coastal_cropped3)
RB8AB8 = np.divide(nir08_cropped3, nir_cropped3)
RB3B2 = np.divide(green_cropped3, blue_cropped3)
RB5B4 = np.divide(rededge1_cropped3, red_cropped3)

In [ ]:
nan_count = np.count_nonzero(np.isnan(RB5B4))

print("Number of NaN values in RB8B2:", nan_count)

In [ ]:
### plot histograms
import matplotlib.pyplot as plt

plt.figure(figsize=(4,3))
plt.hist(RB3B1.flatten(), bins=50, color='blue', alpha=0.5, label='RB3B1')
#plt.hist(red_nir_ratio.flatten(), bins=50, color='red', alpha=0.5, label='Red/NIR Ratio')
# and so on for other ratios

plt.xlabel('Ratio Values')
plt.ylabel('Frequency')
plt.title('Histogram of Ratio Values')
plt.legend()
plt.show()

In [ ]:
print(np.nanmean(RB8B2), np.nanmean(RB8B4), np.nanmean(RB2B1), np.nanmean(RB4B1), np.nanmean(RB8AB8), np.nanmean(RB3B2), np.nanmean(RB5B4), np.nanmean(RB3B1)) 

In [ ]:
print(np.nanstd(RB8B2), np.nanstd(RB8B4), np.nanstd(RB2B1), np.nanstd(RB4B1), np.nanstd(RB8AB8), np.nanstd(RB3B2), np.nanstd(RB5B4), np.nanstd(RB3B1)) 

In [ ]:
### Calculate the NORMALIZED DIFFERENCE between bands

ndB8B2 = (nir_cropped3 - blue_cropped3) / (nir_cropped3 + blue_cropped3)
ndB8B4 = (nir_cropped3 - red_cropped3) / (nir_cropped3 + red_cropped3)
ndB2B1 = (blue_cropped3 - coastal_cropped3) / (blue_cropped3 + coastal_cropped3)
ndB3B1 = (green_cropped3 - coastal_cropped3) / (green_cropped3 + coastal_cropped3)
ndB4B1 = (red_cropped3 - coastal_cropped3) / (red_cropped3 + coastal_cropped3)
ndB8AB8 = (nir08_cropped3 - nir_cropped3) / (nir08_cropped3 + nir_cropped3)
ndB3B2 = (green_cropped3 - blue_cropped3) / (green_cropped3 + blue_cropped3)
ndB5B4 = (rededge1_cropped3 - red_cropped3) / (rededge1_cropped3 + red_cropped3)

In [ ]:
### plot histograms
import matplotlib.pyplot as plt

plt.figure(figsize=(4,3))
plt.hist(ndB5B4.flatten(), bins=50, color='blue', alpha=0.5, label='ndB5B4')
#plt.hist(red_nir_ratio.flatten(), bins=50, color='red', alpha=0.5, label='Red/NIR Ratio')
# and so on for other ratios

#plt.xlabel('Ratio Values')
#plt.ylabel('Frequency')
plt.title('Histogram of Ratio Values')
plt.legend()
plt.show()

In [ ]:
print(np.nanmean(ndB8B2), np.nanmean(ndB8B4), np.nanmean(ndB2B1), np.nanmean(ndB4B1), np.nanmean(ndB8AB8), np.nanmean(ndB3B2), np.nanmean(ndB5B4), np.nanmean(ndB3B1)) 

In [ ]:
print(np.nanstd(ndB8B2), np.nanstd(ndB8B4), np.nanstd(ndB2B1), np.nanstd(ndB4B1), np.nanstd(ndB8AB8), np.nanstd(ndB3B2), np.nanstd(ndB5B4), np.nanstd(ndB3B1)) 

####

# Part 3: Pixel-level classification with your bands

In [ ]:
# Find the minimum non-zero value in the arrays
min_nonzero_value = np.min(np.where(nir_cropped != 0, nir_cropped, np.inf))

# Replace zero values with a fraction of the minimum non-zero value
fraction = 0.1

coastal_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, coastal_cropped)
blue_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, blue_cropped)
green_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, green_cropped)
red_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, red_cropped)
nir_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, nir_cropped)
nir08_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, nir08_cropped)
rededge1_cropped = np.where(nir_cropped == 0, min_nonzero_value * fraction, rededge1_cropped)

In [ ]:
ndB8B2 = (nir_cropped - blue_cropped) / (nir_cropped + blue_cropped)
ndB8B4 = (nir_cropped - red_cropped) / (nir_cropped + red_cropped)
ndB2B1 = (blue_cropped - coastal_cropped) / (blue_cropped + coastal_cropped)
ndB3B1 = (green_cropped - coastal_cropped) / (green_cropped + coastal_cropped)
ndB4B1 = (red_cropped - coastal_cropped) / (red_cropped + coastal_cropped)
ndB8AB8 = (nir08_cropped - nir_cropped) / (nir08_cropped + nir_cropped)
ndB3B2 = (green_cropped - blue_cropped) / (green_cropped + blue_cropped)
ndB5B4 = (rededge1_cropped - red_cropped) / (rededge1_cropped + red_cropped)

In [ ]:
print(ndB8B2.shape)
print(ndB3B1.shape)
print(ndB5B4.shape)
print(ndB3B2.shape)

In [ ]:
# Count the total number of elements in the array
total_elements = np.prod(ndB5B4.shape)

# Count the number of NaN values
nan_count = np.sum(np.isnan(ndB5B4))

# Calculate the percentage of NaN values
percentage_nan = (nan_count / total_elements) * 100

print(f"Percentage of NaN values in ndB5B4: {nan_count}")

In [ ]:
# Substitute NaN values with 0
ndB5B4_no_nan = np.nan_to_num(ndB5B4, nan=0)

In [ ]:
# Create a Boolean mask indicating where final_mask_Posidonia is True and coastal_cropped contains NaN values
nan_mask = np.logical_and(final_mask_water, np.isnan(ndB5B4))

# Count the number of True values in the combined mask
nan_count = np.sum(nan_mask)

# Count the total number of True values in final_mask_Posidonia
total_count = np.sum(final_mask_water)

# Calculate the percentage
percentage_nan_in_Posidonia = (nan_count / total_count) * 100

print(f"Percentage of True values in final_mask_Posidonia that correspond to NaN values in coastal_cropped: {percentage_nan_in_Posidonia:.2f}%")

In [ ]:
import numpy as np

# Check for NaN values in X_train
nan_indices_train = np.isnan(ndB3B2).any(axis=1)
if np.any(nan_indices_train):
    print("NaN values found in X_train")

# Check for infinity values in X_train
inf_indices_train = np.isinf(ndB3B2).any(axis=1)
if np.any(inf_indices_train):
    print("Infinity values found in X_train")

# Check for NaN values in X_test
nan_indices_test = np.isnan(ndB3B2).any(axis=1)
if np.any(nan_indices_test):
    print("NaN values found in X_test")

# Check for infinity values in X_test
inf_indices_test = np.isinf(ndB3B2).any(axis=1)
if np.any(inf_indices_test):
    print("Infinity values found in X_test")


#### first SVM with 3 classes:

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mask_class0 = final_mask_Posidonia
mask_class1 = final_mask_water
mask_class2 = final_mask_Land
 
# Flatten masks
mask_class0_flat = mask_class0.ravel()
mask_class1_flat = mask_class1.ravel()
mask_class2_flat = mask_class2.ravel()

# Initialize y with zeros
y = np.zeros_like(mask_class0_flat, dtype=int)

# Assign class labels based on flattened masks
y[mask_class2_flat] = 2
y[mask_class1_flat] = 1

#y = np.zeros_like(mask_class0_flat, dtype=int)
# Flatten the band
#band_of_interest_flat = ndB3B2.ravel()

# Assign class labels based on the flattened band
#y[band_of_interest_flat == 300] = 3  # Assign placeholder value 300 to a new class, e.g., class 3
#y[mask_class2_flat] = 2
#y[mask_class1_flat] = 1

# Stack bands to create feature vectors
X = np.stack((
    ndB8B2.reshape(-1),
    ndB3B1.reshape(-1),
    ndB5B4_no_nan.reshape(-1),
    ndB3B2.reshape(-1)
), axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Find indices of NaN values in X_train
#nan_indices = np.isnan(X_train).any(axis=1)

# Remove samples with NaN values
#X_train = X_train[~nan_indices]
#y_train = y_train[~nan_indices]

# Train SVM with RBF kernel
clf = svm.SVC(kernel='rbf', decision_function_shape='ovr')  # One-vs-Rest strategy
clf.fit(X_train, y_train)

# Find indices of NaN values in X_train
#nan_indices2 = np.isnan(X_test).any(axis=1)

# Remove samples with NaN values
#X_train = X_test[~nan_indices2]
#y_test = y_test[~nan_indices2]

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
### check if sth goes wrong

import numpy as np

# Check for NaN values in X_train
nan_indices_train = np.isnan(X_train).any(axis=1)
if np.any(nan_indices_train):
    print("NaN values found in X_train")

# Check for infinity values in X_train
inf_indices_train = np.isinf(X_train).any(axis=1)
if np.any(inf_indices_train):
    print("Infinity values found in X_train")

# Check for NaN values in X_test
nan_indices_test = np.isnan(X_test).any(axis=1)
if np.any(nan_indices_test):
    print("NaN values found in X_test")

# Check for infinity values in X_test
inf_indices_test = np.isinf(X_test).any(axis=1)
if np.any(inf_indices_test):
    print("Infinity values found in X_test")


In [ ]:
from sklearn.metrics import accuracy_score

# Assume y_test and y_pred contain the true and predicted labels respectively

# Create a boolean mask for instances where the true labels are class 0
mask_class_0 = (y_test == 0)

# Filter the true labels and predicted labels using the mask for class 0
y_test_class_0 = y_test[mask_class_0]
y_pred_class_0 = y_pred[mask_class_0]

# Calculate accuracy for class 0
accuracy_class_0 = accuracy_score(y_test_class_0, y_pred_class_0)
print("Accuracy for class 0:", accuracy_class_0)

In [ ]:
# Define original_image as your input bands stacked along the first axis
original_image = np.stack((
    ndB8B2,
    ndB3B1,
    ndB5B4,
    ndB3B2
), axis=2)  # Change axis to 2 for RGB image

# Re-predict using the entire dataset X
y_pred_full = clf.predict(X)

# Reshape y_pred_full to match the shape of original_image
predicted_labels_image = y_pred_full.reshape(original_image.shape[:2])

# Plot the segmented image
plt.figure(figsize=(8, 6))
plt.imshow(predicted_labels_image, cmap='viridis')  # You can choose any colormap you prefer
plt.colorbar(label='Class')
plt.title('Segmented Image')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

#### Let's perform XGBOOST with 3 classes

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mask_class0 = final_mask_Posidonia
mask_class1 = final_mask_water
mask_class2 = final_mask_Land

# Flatten masks
mask_class0_flat = mask_class0.ravel()
mask_class1_flat = mask_class1.ravel()
mask_class2_flat = mask_class2.ravel()

# Initialize y with zeros
y = np.zeros_like(mask_class0_flat, dtype=int)

# Assign class labels based on flattened masks
y[mask_class2_flat] = 2
y[mask_class1_flat] = 1

# Stack bands to create feature vectors
X = np.stack((
    ndB8B2.reshape(-1),
    ndB3B1.reshape(-1),
    ndB5B4_no_nan.reshape(-1),
    ndB3B2.reshape(-1)
), axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data into DMatrix format for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameters for XGBoost
param = {
    'max_depth': 3,  # maximum depth of a tree
    'objective': 'multi:softmax',  # multi-class classification
    'num_class': 3,  # number of classes
    'eta': 0.3,  # learning rate
}

# Train the model
num_round = 100  # number of boosting rounds
bst = xgb.train(param, dtrain, num_round)

# Make predictions
preds = bst.predict(dtest)

# Evaluate accuracy
accuracy = accuracy_score(y_test, preds)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import accuracy_score

# Assume y_test and y_pred contain the true and predicted labels respectively

# Create a boolean mask for instances where the true labels are class 0
mask_class_0 = (y_test == 0)

# Filter the true labels and predicted labels using the mask for class 0
y_test_class_0 = y_test[mask_class_0]
y_pred_class_0 = y_pred[mask_class_0]

# Calculate accuracy for class 0
accuracy_class_0 = accuracy_score(y_test_class_0, y_pred_class_0)
print("Accuracy for class 0:", accuracy_class_0)

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
# Define original_image as your input bands stacked along the first axis
original_image = np.stack((
    ndB8B2,
    ndB3B1,
    ndB5B4,
    ndB3B2
), axis=2)  # Change axis to 2 for RGB image

# Re-predict using the entire dataset X
y_pred_full = xgb.predict(X)

# Reshape y_pred_full to match the shape of original_image
predicted_labels_image = y_pred_full.reshape(original_image.shape[:2])

# Plot the segmented image
plt.figure(figsize=(8, 6))
plt.imshow(predicted_labels_image, cmap='viridis')  # You can choose any colormap you prefer
plt.colorbar(label='Class')
plt.title('Segmented Image')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

#### Finally perform Random Forest with 3 classes

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

mask_class0 = final_mask_Posidonia
mask_class1 = final_mask_water
mask_class2 = final_mask_Land

# Flatten masks
mask_class0_flat = mask_class0.ravel()
mask_class1_flat = mask_class1.ravel()
mask_class2_flat = mask_class2.ravel()

# Initialize y with zeros
y = np.zeros_like(mask_class0_flat, dtype=int)

# Assign class labels based on flattened masks
y[mask_class2_flat] = 2
y[mask_class1_flat] = 1

# Stack bands to create feature vectors
#X = np.stack((
#    ndB8B2.reshape(-1),
#    ndB3B1.reshape(-1),
#    ndB5B4.reshape(-1),
#    ndB3B2.reshape(-1)
#), axis=1)

#X = np.stack((
#    nir_cropped.reshape(-1),
#    blue_cropped.reshape(-1),
#    red_cropped.reshape(-1),
#    green_cropped.reshape(-1),
#    coastal_cropped.reshape(-1)
#), axis=1)

X = np.stack((
    blue_cropped.reshape(-1),
    red_cropped.reshape(-1),
    green_cropped.reshape(-1),
), axis=1)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred = rf_classifier.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


In [ ]:
from sklearn.metrics import accuracy_score

# Assume y_test and y_pred contain the true and predicted labels respectively

# Create a boolean mask for instances where the true labels are class 0
mask_class_0 = (y_test == 0)

# Filter the true labels and predicted labels using the mask for class 0
y_test_class_0 = y_test[mask_class_0]
y_pred_class_0 = y_pred[mask_class_0]

# Calculate accuracy for class 0
accuracy_class_0 = accuracy_score(y_test_class_0, y_pred_class_0)
print("Accuracy for class 0:", accuracy_class_0)

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
# Define original_image as your input bands stacked along the first axis
original_image = np.stack((
    ndB8B2,
    ndB3B1,
    ndB5B4,
    ndB3B2
), axis=2)  # Change axis to 2 for RGB image

# Re-predict using the entire dataset X
y_pred_full = rf_classifier.predict(X)

# Reshape y_pred_full to match the shape of original_image
predicted_labels_image = y_pred_full.reshape(original_image.shape[:2])

# Plot the segmented image
plt.figure(figsize=(8, 6))
plt.imshow(predicted_labels_image, cmap='viridis')  # You can choose any colormap you prefer
plt.colorbar(label='Class')
plt.title('Segmented Image')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

#### And, additionally, do K-means clustering

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

# Assuming you have numpy arrays for each band
# Let's say band1, band2, ..., bandN are your individual numpy arrays

# Stack bands to create feature vectors
#X = np.stack((
#    ndB8B2.reshape(-1),
#    ndB3B1.reshape(-1),
#    ndB5B4_no_nan.reshape(-1),
#    ndB3B2.reshape(-1)
#), axis=1)

X = np.stack((
    nir_cropped.reshape(-1),
    blue_cropped.reshape(-1),
    red_cropped.reshape(-1),
    green_cropped.reshape(-1),
    coastal_cropped.reshape(-1)
), axis=1)

# Perform K-means clustering with K=3
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X)

# Get the labels of the clusters
labels = kmeans.labels_

In [ ]:
#original_image = np.stack((
#    ndB8B2,
#    ndB3B1,
#    ndB5B4,
#    ndB3B2
#), axis=2) 

original_image = np.stack((
    nir_cropped,
    blue_cropped,
    red_cropped,
    green_cropped,
    coastal_cropped
), axis=2) 

segmented_image = labels.reshape(original_image.shape[:2])

# Plot the segmented image
plt.figure(figsize=(8, 6))
plt.imshow(segmented_image, cmap='viridis')  # You can choose any colormap you prefer
plt.colorbar(label='Cluster')
plt.title('Segmented Image (K-Means)')
plt.xlabel('Width')
plt.ylabel('Height')
plt.show()

In [ ]:
mask_class0 = final_mask_Posidonia
mask_class1 = final_mask_water
mask_class2 = final_mask_Land

In [ ]:
# Count unique values in segmented_image
unique_values = np.unique(segmented_image)

# Initialize a dictionary to store percentages
percentage_dict = {}

# Iterate over unique values
for value in unique_values:
    # Mask for pixels with the current value in segmented_image
    value_mask = segmented_image == value
    
    # Calculate percentages within each mask
    percentages = []
    for mask in [mask_class0, mask_class1, mask_class2]:
        # Count pixels within the current mask and with the current value
        pixels_in_mask = np.sum(np.logical_and(value_mask, mask))
        total_pixels = np.sum(mask)  # Total number of pixels in the mask
        percentage = (pixels_in_mask / total_pixels) * 100
        
        percentages.append(percentage)
    
    # Store percentages in the dictionary
    percentage_dict[value] = percentages

# Create a table to display the percentages
print("Unique Value\tPercentage in Mask 0\tPercentage in Mask 1\tPercentage in Mask 2")
for value, percentages in percentage_dict.items():
    print(f"{value}\t\t{percentages[0]:.2f}%\t\t\t{percentages[1]:.2f}%\t\t\t{percentages[2]:.2f}%")
